In [3]:
from langchain_openai import OpenAI, ChatOpenAI
import os
from datasets import load_dataset
from pinecone import Pinecone
from tqdm.auto import tqdm
from langchain.chains import LLMMathChain
from langchain.agents import Tool, initialize_agent
from langchain.callbacks import StreamingStdOutCallbackHandler
from warnings import filterwarnings

filterwarnings("ignore")
os.environ["OPENAI_API_KEY"] = ""

## Important Note: Not All LLMs support streaming

In [4]:
llm = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo",
                 streaming = True,
                 callbacks = [StreamingStdOutCallbackHandler()])

In [5]:
llm.invoke("hi, how are you?")

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?")

In [8]:
from langchain.schema import HumanMessage

messages = [HumanMessage(content = "Tell me a long story")]
llm.invoke(messages)

Once upon a time, in a small village nestled in the rolling hills of the countryside, there lived a young girl named Elara. Elara was known throughout the village for her kindness and compassion towards others. She spent her days helping the elderly, caring for the sick, and tending to the animals in need.

One day, as Elara was walking through the forest on the outskirts of the village, she stumbled upon a wounded deer. The poor creature had been caught in a hunter's trap and was in desperate need of help. Without hesitation, Elara knelt down beside the deer and gently freed it from the trap. She then tended to its wounds, using herbs and poultices to ease its pain.

As the days passed, Elara continued to care for the deer, feeding it and nursing it back to health. The two formed a deep bond, and the deer began to follow Elara wherever she went. The villagers marveled at the sight of the girl and the deer walking side by side, and many believed that Elara possessed a special gift for 

AIMessage(content="Once upon a time, in a small village nestled in the rolling hills of the countryside, there lived a young girl named Elara. Elara was known throughout the village for her kindness and compassion towards others. She spent her days helping the elderly, caring for the sick, and tending to the animals in need.\n\nOne day, as Elara was walking through the forest on the outskirts of the village, she stumbled upon a wounded deer. The poor creature had been caught in a hunter's trap and was in desperate need of help. Without hesitation, Elara knelt down beside the deer and gently freed it from the trap. She then tended to its wounds, using herbs and poultices to ease its pain.\n\nAs the days passed, Elara continued to care for the deer, feeding it and nursing it back to health. The two formed a deep bond, and the deer began to follow Elara wherever she went. The villagers marveled at the sight of the girl and the deer walking side by side, and many believed that Elara posses

In [14]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools, AgentType, initialize_agent
memory = ConversationBufferWindowMemory(memory_key = "chat_history",
                                        k = 5,
                                        return_messages= True,
                                        output_key= "output")

tools = load_tools(["llm-math"], llm = llm)

agent = initialize_agent(agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
                         tools = tools,
                         llm = llm,
                         memory = memory,
                         verbose = True,
                         max_iterations = 3,
                         early_stopping_method = "generate",
                         return_intermediate_steps = False)

In [15]:
agent.invoke("Hi, How are you?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"
}
``````json
{
    "action": "Final Answer",
    "action_input": "I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"
}
```

> Finished chain.


{'input': 'Hi, How are you?',
 'chat_history': [],
 'output': "I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"}

In [16]:
agent.invoke("Can you tell me the square root of 459")



> Entering new AgentExecutor chain...
```json
{
    "action": "Calculator",
    "action_input": "square root of 459"
}
``````json
{
    "action": "Calculator",
    "action_input": "square root of 459"
}
``````text
459**0.5
```
...numexpr.evaluate("459**0.5")...

Observation: Answer: 21.42428528562855
Thought:```json
{
    "action": "Final Answer",
    "action_input": "21.42428528562855"
}
``````json
{
    "action": "Final Answer",
    "action_input": "21.42428528562855"
}
```

> Finished chain.


{'input': 'Can you tell me the square root of 459',
 'chat_history': [HumanMessage(content='Hi, How are you?'),
  AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?")],
 'output': '21.42428528562855'}

In [17]:
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler

In [18]:
agent.agent.llm_chain.llm.callbacks = [FinalStreamingStdOutCallbackHandler()]

In [19]:
agent.invoke("What is the square root of 5457?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Calculator",
    "action_input": "square root of 5457"
}
```
Observation: Answer: 73.87151006985034
Thought:```json
{
    "action": "Final Answer",
    "action_input": "73.87151006985034"
}
```

> Finished chain.


{'input': 'What is the square root of 5457?',
 'chat_history': [HumanMessage(content='Hi, How are you?'),
  AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"),
  HumanMessage(content='Can you tell me the square root of 459'),
  AIMessage(content='21.42428528562855')],
 'output': '73.87151006985034'}

In [20]:
agent.agent.llm_chain.llm.callbacks = [FinalStreamingStdOutCallbackHandler(
    answer_prefix_tokens = ["Final", "Answer"]
)]

In [21]:
agent.invoke("What is the square root of 5457?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Calculator",
    "action_input": "square root of 5457"
}
```
Observation: Answer: 73.87151006985034
Thought:",
    "action_input": "73.87151006985034"
}
``````json
{
    "action": "Final Answer",
    "action_input": "73.87151006985034"
}
```

> Finished chain.


{'input': 'What is the square root of 5457?',
 'chat_history': [HumanMessage(content='Hi, How are you?'),
  AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"),
  HumanMessage(content='Can you tell me the square root of 459'),
  AIMessage(content='21.42428528562855'),
  HumanMessage(content='What is the square root of 5457?'),
  AIMessage(content='73.87151006985034')],
 'output': '73.87151006985034'}

In [22]:
import sys

class CallbackHandler(StreamingStdOutCallbackHandler):
    def __init__(self):
        self.content: str = ""
        self.final_answer: bool = False

    def on_llm_new_token(self, token: str, **kwargs: any) -> None:
        self.content += token
        if "Final Answer" in self.content:
            self.final_answer = True
            self.content = ""
        
        if self.final_answer:
            if '"action_input": "' in self.content:
                if token not in ["}"]:
                    sys.stdout.write(token)
                    sys.stdout.flush()

In [23]:
agent.agent.llm_chain.llm.callbacks = [CallbackHandler()]

In [24]:
agent.invoke("What is the cube root of 15472?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Calculator",
    "action_input": "cube root of 15472"
}
```
Observation: Answer: 24.91813219917046
Thought: "73.87151006985034"
}
``````json
{
    "action": "Final Answer",
    "action_input": "73.87151006985034"
}
```

> Finished chain.


{'input': 'What is the cube root of 15472?',
 'chat_history': [HumanMessage(content='Hi, How are you?'),
  AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?"),
  HumanMessage(content='Can you tell me the square root of 459'),
  AIMessage(content='21.42428528562855'),
  HumanMessage(content='What is the square root of 5457?'),
  AIMessage(content='73.87151006985034'),
  HumanMessage(content='What is the square root of 5457?'),
  AIMessage(content='73.87151006985034')],
 'output': '73.87151006985034'}

In [26]:
agent.invoke("Tell me a long story")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final "I'm happy to tell you a story! Once upon a time, in a faraway land, there lived a wise old wizard..."
}
``````json
{
    "action": "Final Answer",
    "action_input": "I'm happy to tell you a story! Once upon a time, in a faraway land, there lived a wise old wizard..."
}
```

> Finished chain.


{'input': 'Tell me a long story',
 'chat_history': [HumanMessage(content='Can you tell me the square root of 459'),
  AIMessage(content='21.42428528562855'),
  HumanMessage(content='What is the square root of 5457?'),
  AIMessage(content='73.87151006985034'),
  HumanMessage(content='What is the square root of 5457?'),
  AIMessage(content='73.87151006985034'),
  HumanMessage(content='What is the cube root of 15472?'),
  AIMessage(content='73.87151006985034'),
  HumanMessage(content='Tell me a long story?'),
  AIMessage(content="I'm happy to tell you a story! Once upon a time, in a faraway land, there lived a wise old wizard...")],
 'output': "I'm happy to tell you a story! Once upon a time, in a faraway land, there lived a wise old wizard..."}

Refer demo_api.py for Fast API implementation

API Running on localhost fastAPI - 
Command used on CMD--> uvicorn demo_api:app --reload

In [34]:
import requests 

res = requests.get("http://localhost:8000/health")
res.json()

{'Status': 'OK'}

In [35]:
res = requests.get("http://localhost:8000/chat",
                   json = {"text": "hello there!"})
res

<Response [200]>

In [36]:
res.json()

{'input': 'hello there!',
 'chat_history': [],
 'output': 'Hello! How can I assist you today?'}

In [38]:
def get_stream(query: str):
    s = requests.Session()
    with s.get(
        "http://localhost:8000/chat",
        stream = True,
        json = {"text": query}
    ) as r:
        for line in r.iter_content():
            print(line.decode("utf-8"), end = "")

In [40]:
get_stream("Tell me a long story")

```json
{
    "action": "Final Answer",
    "action_input": "Once upon a time, in a faraway land, there was a brave knight named Sir Lancelot. He embarked on a quest to rescue a princess from the clutches of an evil dragon. Along the way, he faced many challenges and obstacles, but with courage and determination, he overcame them all. In the end, Sir Lancelot defeated the dragon, saved the princess, and brought peace to the kingdom. And they all lived happily ever after."
}
```